In [26]:
import pandas as pd

## Import des Données

In [27]:
path1 = "Données/consommation-annuelle-par-commune-new.csv"
path2 = "Données/MDB-INSEE-V2.xls"

In [28]:
conso_annuelle = pd.read_csv(path1, encoding = 'latin', sep=";")
btp_etat = pd.read_excel(path2)


WARNING *** file size (57125979) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero


In [68]:
btp_etat["CODGEO"].unique()

array(['01001', '01002', '01004', ..., '97422', '97423', '97424'],
      dtype=object)

In [32]:
list(btp_etat.columns)

['CODGEO',
 'Nb Pharmacies et parfumerie',
 'Dynamique Entrepreneuriale',
 'Dynamique Entrepreneuriale Service et Commerce',
 'Synergie Médicale COMMUNE',
 'Orientation Economique',
 'Indice Fiscal Partiel',
 'Score Fiscal',
 'Indice Evasion Client',
 'Score Evasion Client',
 'Indice Synergie Médicale',
 'Score Synergie Médicale',
 'SEG Croissance POP',
 'LIBGEO',
 'REG',
 'DEP',
 'Nb Omnipraticiens BV',
 'Nb Infirmiers Libéraux BV',
 'Nb dentistes Libéraux BV',
 'Nb pharmaciens Libéraux BV',
 'Densité Médicale BV',
 'Score équipement de santé BV',
 'Indice Démographique',
 'Score Démographique',
 'Indice Ménages',
 'Score Ménages',
 'Population',
 'Evolution Population',
 'Evolution Pop %',
 'Nb Ménages',
 'Nb Résidences Principales',
 'Nb propriétaire',
 'Nb Logement',
 'Nb Résidences Secondaires',
 'Nb Log Vacants',
 'Nb Occupants Résidence Principale',
 'Nb Femme',
 'Nb Homme',
 'Nb Mineurs',
 'Nb Majeurs',
 'Nb Etudiants',
 'Nb Entreprises Secteur Services',
 'Nb Entreprises Secte

In [33]:
all_interesting_features = btp_etat.columns

In [30]:
conso_annuelle

,territoire,commune,annee,geo_shape,code_insee,secteur,consommation_mwh,nombre_de_pds,geo_point_2d
0,Corse,Afa,2012,"{""type"": ""Polygon"", ""coordinates"": [[[8.784623...",2A001,Non affectÃ©,17364.0,NaN,"41.9839501985,8.79834460109"
1,Corse,Albitreccia,2012,"{""type"": ""Polygon"", ""coordinates"": [[[8.912546...",2A008,Non affectÃ©,9067.0,NaN,"41.860583203,8.8773015755"
2,Corse,Altiani,2012,"{""type"": ""Polygon"", ""coordinates"": [[[9.291797...",2B012,Non affectÃ©,456.0,NaN,"42.2199015502,9.27896340729"
3,Corse,Ampriani,2012,"{""type"": ""Polygon"", ""coordinates"": [[[9.357045...",2B015,Non affectÃ©,82.0,NaN,"42.2542599711,9.35682808827"
4,Corse,Antisanti,2012,"{""type"": ""Polygon"", ""coordinates"": [[[9.316641...",2B016,Non affectÃ©,4972.0,NaN,"42.1565359376,9.3833327974"
...,...,...,...,...,...,...,...,...,...
10323,RÃ©union,Saint-Philippe,2019,"{""type"": ""Polygon"", ""coordinates"": [[[55.67876...",97417,RÃ©sidentiel,6356.0,2082.0,"-21.303948518,55.7450019415"
10324,RÃ©union,Sainte-Marie,2019,"{""type"": ""Polygon"", ""coordinates"": [[[55.54069...",97418,Tertiaire,72334.0,1065.0,"-20.946812748,55.5307201449"
10325,RÃ©union,Sainte-Rose,2019,"{""type"": ""Polygon"", ""coordinates"": [[[55.67876...",97419,Agriculture,121.0,14.0,"-21.1923212248,55.7545038711"
10326,RÃ©union,Sainte-Rose,2019,"{""type"": ""Polygon"", ""coordinates"": [[[55.67876...",97419,Industrie,481.0,18.0,"-21.1923212248,55.7545038711"


### Jointure par code insee

In [62]:
path = "correspondance-code-insee-code-postal.csv"
df_insee_cp = pd.read_csv(path, encoding = 'latin', sep=";").rename(columns={"Code Postal":"CODGEO"})
df_insee_cp[["Code INSEE","CODGEO"]]

,Code INSEE,CODGEO
0,59276,59287
1,88128,88210
2,57538,57170
3,54459,54630
4,35022,35190
...,...,...
36737,72103,72150
36738,57312,57220
36739,32383,32230
36740,17278,17600


In [52]:
df_insee_cp["CP"][0]

'59287'

In [61]:
btp_etat["R"][500]

'02'

In [121]:
join = btp_etat.join(df_insee_cp[["Code INSEE","CODGEO"]].set_index('CODGEO'), on='CODGEO')
join_insee_cp = join[~join["Code INSEE"].isna()]

In [122]:
conso_communes = conso_annuelle.rename(columns={"code_insee":"Code INSEE"}).join(join_insee_cp.set_index("Code INSEE"), on="Code INSEE")

## Regression

In [123]:
Y = conso_communes["consommation_mwh"]
X = conso_communes.drop("consommation_mwh",axis=1)

In [124]:
all_interesting_features = ["Nb Pharmacies et parfumerie","Dynamique Entrepreneuriale", "Dynamique Entrepreneuriale Service et Commerce", "Score Fiscal", "Score Synergie Médicale","Nb Omnipraticiens BV", "Score Démographique", "Score Ménages","Population","Evolution Population"]

In [125]:
for c in X.columns:
    if "Nb" in c and c not in all_interesting_features:
        all_interesting_features.append(c)

In [126]:
for c in all_interesting_features:
    X[c] = pd.to_numeric(X[c], downcast='float')

In [127]:
X = X[all_interesting_features].fillna(0)

In [128]:
from sklearn import datasets, linear_model
from sklearn.model_selection import cross_validate
from sklearn.metrics import make_scorer
from sklearn.metrics import confusion_matrix

lasso = linear_model.Lasso()

In [129]:
scores = cross_validate(lasso, X, Y, cv=3,
...                         scoring=('r2', 'neg_mean_squared_error'),
...                         return_train_score=True)
print(scores['test_neg_mean_squared_error'])
print(scores['train_r2'])

/home/rym/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 3131799583358.3716, tolerance: 671531045.5716113
  positive)


[-2.24724606e+08 -8.72857654e+07 -1.98360602e+09]
[0.06733874 0.06026607 0.        ]


/home/rym/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 3384761150855.975, tolerance: 720315641.8705244
  positive)
